In [7]:
import sys; sys.path.append('..')
from utils import stop_grad

: 

In [1]:
import jittor as jt
import sys; sys.path.append('..')
from model_jittor.ldm.ema import *
from model_jittor.ldm.openai_model import UNetModel
import matplotlib.pyplot as plt
from model_jittor.ldm.ema import *
jt.flags.use_cuda = True

%load_ext autoreload
%autoreload 2

[i 0710 04:59:36.930645 84 log.cc:351] Load log_sync: 1
[i 0710 04:59:37.022817 84 compiler.py:953] Jittor(1.3.4.15) src: /root/miniconda3/envs/jittor/lib/python3.7/site-packages/jittor
[i 0710 04:59:37.047958 84 compiler.py:954] g++ at /usr/bin/g++(7.5.0)
[i 0710 04:59:37.051512 84 compiler.py:955] cache_path: /root/.cache/jittor/jt1.3.4/g++7.5.0/py3.7.0/Linux-4.4.0-11x56/IntelRXeonRGolxda/default
[i 0710 04:59:37.086602 84 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.2.152) at /usr/local/cuda/bin/nvcc.
[i 0710 04:59:37.106921 84 __init__.py:411] Found addr2line(2.30) at /usr/bin/addr2line.
[i 0710 04:59:37.347848 84 compiler.py:1008] cuda key:cu11.2.152_sm_75
[i 0710 04:59:37.707753 84 __init__.py:227] Total mem: 376.57GB, using 16 procs for compiling.
[i 0710 04:59:37.903816 84 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0710 04:59:38.214966 84 init.cc:62] Found cuda archs: [75,]
[i 0710 04:59:39.086047 84 __init__.py:411] Found mpicc(2.1.1) at /usr/bin/mpicc.
[i 0710 04

In [2]:
model = UNetModel()
ema_model = EMAModel(model)

In [3]:
model.state_dict()['time_embed.0.weight'][0][:5]


Compiling Operators(9/9) used: 3.32s eta:    0s 


jt.Var([ 0.00072185 -0.07397059 -0.0177581   0.00604334  0.02918333], dtype=float32)

In [4]:
ema_model.averaged_model.state_dict()['time_embed.0.weight'][0][:5]

jt.Var([ 0.00072185 -0.07397059 -0.0177581   0.00604334  0.02918333], dtype=float32)

In [5]:
input = jt.rand(2, 6, 32, 32).stop_grad()
t = jt.randint(0, 1000, (2,))
target = jt.rand(2, 3, 32, 32).stop_grad()

optimizer_jittor = jt.optim.Adam(
    model.parameters(), 
    lr=1e-5, 
)

losses_jittor = []
model.train() # NOTE: do not forget
for i in range(10):
    output = model(input, t)
    loss_jittor = nn.mse_loss(output , target)
    optimizer_jittor.step(loss_jittor)
    losses_jittor.append(loss_jittor.item())
    ema_model.step(model)
    print(ema_model.averaged_model.state_dict()['time_embed.0.weight'][0][:5])
    print(loss_jittor.data)


Compiling Operators(40/40) used: 4.31s eta:    0s 
[w 0710 05:00:00.776390 84 cuda_device_allocator.cc:29] Unable to alloc cuda device memory, use unify memory instead. This may cause low performance.
[i 0710 05:00:00.777914 84 cuda_device_allocator.cc:31] 
=== display_memory_info ===
 total_cpu_ram: 376.6GB total_device_ram: 23.65GB
 hold_vars: 1091 lived_vars: 13056 lived_ops: 13086
 name: sfrl is_device: 1 used: 1.671GB(99.1%) unused: 16.25MB(0.941%) total: 1.687GB
 name: sfrl is_device: 1 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: sfrl is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: sfrl is_device: 0 used: 1.291MB(64.6%) unused:   726KB(35.4%) total:     2MB
 name: temp is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: temp is_device: 1 used:     0 B(0%) unused: 4.291GB(100%) total: 4.291GB
 cpu&gpu:  5.98GB gpu: 5.978GB cpu:     2MB
 free: cpu(69.13GB) gpu(3.473GB)


Compiling Operators(45/53) used

jt.Var([ 0.00073185 -0.07398058 -0.0177681   0.00603335  0.02919333], dtype=float32)
[0.445877]



Compiling Operators(2/5) used: 4.31s eta: 6.47s 5/5) used: 5.33s eta:    0s 


jt.Var([ 0.00074095 -0.07398882 -0.01777732  0.00602386  0.02920234], dtype=float32)
[0.27721608]



Compiling Operators(5/5) used: 4.35s eta:    0s 


jt.Var([ 0.0007458  -0.07399333 -0.01778222  0.00601886  0.02920716], dtype=float32)
[0.20266539]
jt.Var([ 0.00075045 -0.07399783 -0.01778688  0.0060142   0.0292118 ], dtype=float32)
[0.16479117]
jt.Var([ 0.00075483 -0.07400221 -0.01779124  0.0060099   0.02921619], dtype=float32)
[0.14159963]
jt.Var([ 0.00075893 -0.07400645 -0.0177953   0.00600598  0.02922031], dtype=float32)
[0.12693618]
jt.Var([ 0.00076275 -0.07401053 -0.01779905  0.0060024   0.02922417], dtype=float32)
[0.11754528]
jt.Var([ 0.00076629 -0.07401441 -0.01780251  0.00599916  0.02922776], dtype=float32)
[0.11110616]
jt.Var([ 0.00076957 -0.07401809 -0.01780569  0.00599622  0.02923109], dtype=float32)
[0.10604455]
jt.Var([ 0.00077259 -0.07402157 -0.01780861  0.00599356  0.02923417], dtype=float32)
[0.10147465]
